In [2]:
import pandas as pd
import numpy as np
import os
import sys
import gzip
import shutil
import json
import re
import tmalign_mapping
import Bio.PDB as bpdb
from Bio import Align

In [3]:
filepath_sample = "/nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/3hqa.cif"
parser = bpdb.MMCIFParser()

In [4]:
structure=parser.get_structure("3hqa", filepath_sample)
structure_2 = parser.get_structure("3hqb", "/nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/3hqb.cif")

In [5]:
res_list= bpdb.Selection.unfold_entities(structure, "C")
print(res_list)

[<Chain id=A>, <Chain id=B>]


In [6]:
structure[0]['B'].get_residues()
res_list= bpdb.Selection.unfold_entities(structure[0]['B'], "R")
print(res_list)
# print(structure[0]['B'][2])
print(structure[0]['A'][1])

[<Residue GLN het=  resseq=3 icode= >, <Residue LYS het=  resseq=4 icode= >, <Residue LYS het=  resseq=5 icode= >, <Residue ILE het=  resseq=6 icode= >, <Residue GLU het=  resseq=7 icode= >, <Residue GLU het=  resseq=8 icode= >, <Residue ILE het=  resseq=9 icode= >, <Residue ALA het=  resseq=10 icode= >, <Residue ALA het=  resseq=11 icode= >, <Residue LYS het=  resseq=12 icode= >, <Residue TYR het=  resseq=13 icode= >, <Residue LYS het=  resseq=14 icode= >, <Residue HIS het=  resseq=15 icode= >, <Residue SER het=  resseq=16 icode= >, <Residue VAL het=  resseq=17 icode= >, <Residue VAL het=  resseq=18 icode= >, <Residue LYS het=  resseq=19 icode= >, <Residue LYS het=  resseq=20 icode= >, <Residue CYS het=  resseq=21 icode= >, <Residue CYS het=  resseq=22 icode= >, <Residue TYR het=  resseq=23 icode= >, <Residue ASP het=  resseq=24 icode= >, <Residue GLY het=  resseq=25 icode= >, <Residue ALA het=  resseq=26 icode= >, <Residue CYS het=  resseq=27 icode= >, <Residue VAL het=  resseq=28 ic

In [7]:
ppb= bpdb.PPBuilder()
polypeptides = ppb.build_peptides(structure)
for pp in polypeptides:
    print(pp.get_sequence())
    print(pp[0].get_id())
    print(pp[-1].get_id())
    print(len(pp))

MLQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANIS
(' ', 1, ' ')
(' ', 66, ' ')
66
QKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANIS
(' ', 3, ' ')
(' ', 66, ' ')
64


In [8]:
polypeptides_2 = ppb.build_peptides(structure_2)
for x in ppb.build_peptides(structure_2):
    print(x.get_sequence())
    print(x[0].get_id())
    print(x[-1].get_id())
    print(len(x))


MLQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA
(' ', 1, ' ')
(' ', 63, ' ')
63
LQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA
(' ', 2, ' ')
(' ', 63, ' ')
62


In [9]:
print(str(polypeptides[1].get_sequence()))
print(str(polypeptides_2[1].get_sequence()))

QKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANIS
LQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA


In [10]:
print(structure[0]['A'][64])
print(structure_2[0]['B'][2])

<Residue ASN het=  resseq=64 icode= >
<Residue LEU het=  resseq=2 icode= >


In [72]:
# structure_2[0]['B'].detach_child((' ', 2, ' '))

In [11]:
class GlySelect(bpdb.Select):
    def accept_residue(self, residue):
        if residue.id[1] < 64 and residue.parent.id == 'A':
            return 1
        elif residue.id[1] <=63 and residue.parent.id == 'B':
            return 1
        else:
            return 0

In [12]:
io=bpdb.PDBIO()
io.set_structure(structure)
io.save("trial.pdb", GlySelect())

In [13]:
parser_2 = bpdb.PDBParser()
new_str = parser_2.get_structure("lol", "trial.pdb")
for x in ppb.build_peptides(new_str):
    print(x.get_sequence())
    print(x[0].get_id())

MLQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA
(' ', 1, ' ')
QKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA
(' ', 3, ' ')


In [14]:
res_list= bpdb.Selection.unfold_entities(new_str, "R")
print(res_list)

[<Residue MET het=  resseq=1 icode= >, <Residue LEU het=  resseq=2 icode= >, <Residue GLN het=  resseq=3 icode= >, <Residue LYS het=  resseq=4 icode= >, <Residue LYS het=  resseq=5 icode= >, <Residue ILE het=  resseq=6 icode= >, <Residue GLU het=  resseq=7 icode= >, <Residue GLU het=  resseq=8 icode= >, <Residue ILE het=  resseq=9 icode= >, <Residue ALA het=  resseq=10 icode= >, <Residue ALA het=  resseq=11 icode= >, <Residue LYS het=  resseq=12 icode= >, <Residue TYR het=  resseq=13 icode= >, <Residue LYS het=  resseq=14 icode= >, <Residue HIS het=  resseq=15 icode= >, <Residue SER het=  resseq=16 icode= >, <Residue VAL het=  resseq=17 icode= >, <Residue VAL het=  resseq=18 icode= >, <Residue LYS het=  resseq=19 icode= >, <Residue LYS het=  resseq=20 icode= >, <Residue CYS het=  resseq=21 icode= >, <Residue CYS het=  resseq=22 icode= >, <Residue TYR het=  resseq=23 icode= >, <Residue ASP het=  resseq=24 icode= >, <Residue GLY het=  resseq=25 icode= >, <Residue ALA het=  resseq=26 icod

In [15]:
class GllolSelect(bpdb.Select):
    def accept_residue(self, residue):
        if residue.parent.id == 'A':
            return 1
        elif residue.id[1] > 2 and residue.parent.id == 'B':
            return 1
        else:
            return 0

In [16]:
io=bpdb.PDBIO()
io.set_structure(structure_2)
io.save("trial_2.pdb", GllolSelect())

In [17]:
new_str = parser_2.get_structure("lol2", "trial_2.pdb")
for x in ppb.build_peptides(new_str):
    print(x.get_sequence())
    print(x[0].get_id())

MLQKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA
(' ', 1, ' ')
QKKIEEIAAKYKHSVVKKCCYDGACVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRA
(' ', 3, ' ')


# Let's do the alignment using biopython

In [22]:
aligner = Align.PairwiseAligner()
aligner.mode = 'local'
aligner.open_gap_score = -11
aligner.extend_gap_score = -1
aligner.substitution_matrix = Align.substitution_matrices.load("BLOSUM62")
alignments = aligner.align(str(polypeptides[1].get_sequence()), str(polypeptides_2[1].get_sequence()))
alignments = list(alignments)
print(alignments[0].aligned)

(((0, 61),), ((1, 62),))


In [36]:
tmalign_mapping.unzip_cif_folder("/nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN")
structure_3 = parser.get_structure("1cfa", "/nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/1cfa.cif")
structure_4 = parser.get_structure("4p39", "/nfs/turbo/lsa-tewaria/uniprot/CO5_HUMAN/4p39.cif")

/home/srayan/miniconda3/envs/idp/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2143.
  warnings.warn(
/home/srayan/miniconda3/envs/idp/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2154.
  warnings.warn(
/home/srayan/miniconda3/envs/idp/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 2158.
  warnings.warn(
/home/srayan/miniconda3/envs/idp/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 2161.
  warnings.warn(


In [35]:
print(structure_3[0]["B"][72])
polypeptides_3 = ppb.build_peptides(structure_3)
for x in ppb.build_peptides(structure_3[13]):
    print(x.get_sequence())
    print(x[0].get_id())
    print(x[-1].get_id())

<Residue CYS het=  resseq=72 icode= >
MLQKKIEEIAAKYKHSVVKKCCYDGASVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANISHKDMC
(' ', 1, ' ')
(' ', 71, ' ')
CLG
(' ', 72, ' ')
(' ', 74, ' ')


In [29]:
for model in structure_3:
    print(model.id)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [25]:
alignments = aligner.align(str(polypeptides[0].get_sequence()), str(polypeptides_3[0].get_sequence()))
alignments = list(alignments)
alignment_1, alignment_2 = alignments[0].aligned
alignment_1, alignment_2 = alignment_1[0], alignment_2[0]

In [26]:
for idx, residue in enumerate(structure_3[0]["A"].get_residues()):
    if idx not in range(alignment_2[0], alignment_2[1]):
        structure_3[0]["A"].detach_child(residue.id)

In [27]:
for x in ppb.build_peptides(structure_4):
    print(x.get_sequence())
    print(x[0].get_id())
    print(x[-1].get_id())

STLQKKIEEIAAKYKHSVVKKCCYDGARVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANISFK
(' ', 677, ' ')
(' ', 745, ' ')
STLQKKIEEIAAKYKHSVVKKCCYDGARVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANISFK
(' ', 677, ' ')
(' ', 745, ' ')
TLQKKIEEIAAKYKHSVVKKCCYDGARVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANISFKR
(' ', 678, ' ')
(' ', 746, ' ')
TLQKKIEEIAAKYKHSVVKKCCYDGARVNNDETCEQRAARISLGPRCIKAFTECCVVASQLRANISFK
(' ', 678, ' ')
(' ', 745, ' ')
